Please remember to update the token. For multiplayer to work, each player needs a different token. Please feel free to use multiple tokens in the same group.

In [2]:
#Update your token
STUDENT_TOKEN = 'Juan Rubiales'

There should be no need to change the code below.

In [3]:
## ignore this code, just used for submission
import requests
import pprint
import json
import random
import time
from copy import copy, deepcopy

class Game:
  def __init__(self, state, status, player):
    self.state = state
    self.status = status
    self.player = player

  def is_waiting(self):
    return self.status == 'waiting'

  def is_end(self):
    return self.status == 'complete'
  
  def get_board(self):
    return json.loads(self.state)

  def get_winner(self):
    return None

  def actions(self):
    return []

  def print(self):
    print(self.state)

def new_game(game_type, multi_player = False):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/new-game?TOKEN=%s&game-type=%s&multi-player=%s' % (STUDENT_TOKEN, game_type, 'True' if multi_player else 'False'))
    if r.status_code == 200:
      return r.json()['game-id']
    print(r.content)

def join_game(game_type, game_id):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/join-game?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return r.json()['player']
    print(r.content)

def game_state(game_type, game_id, GameClass):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/game-state?TOKEN=%s&game-type=%s&game-id=%s' % (STUDENT_TOKEN, game_type, game_id))
    if r.status_code == 200:
      return GameClass(r.json()['state'], r.json()['status'], r.json()['player'])
    print(r.content)

def update_game(game_type, game_id, player, move):
  for _ in range(10):
    r = requests.get('https://emarchiori.eu.pythonanywhere.com/update-game?TOKEN=%s&game-type=%s&game-id=%s&player=%s&move=%s' % (STUDENT_TOKEN, game_type, game_id, player, move))
    if r.status_code == 200:
      return r.content
    print(r.content)

def game_loop(solver, GameClass, game_type, multi_player = False, id = None):
  while id == None:
    print('Creating new game...')
    id = new_game(game_type, multi_player)

  print('Joining game with id: %s' % id)
  player = join_game(game_type, id)

  print('Playing as %s' % player)

  game = game_state(game_type, id, GameClass)
  print('Waiting for the other player to join...')
  while game.is_waiting():
    time.sleep(10)
    game = game_state(game_type, id, GameClass)

  while True:
    game = game_state(game_type, id, GameClass)
    game.print_game()
    if game.is_end():
      if game.player == '-':
        print('Server says there is a draw')
      else:
        print('Server says your agent ' + ('WON' if game.player == player else 'LOST'))
      return
    if game.player == player:
      print('Making next move...')
      move = solver(game)
      print(move)
      update_result = update_game(game_type, id, player, json.dumps(move))
      print(update_result)
    else:
      time.sleep(2)

There is almost certainly no need to change this code, but if you must you can.

In [4]:
from functools import reduce
from copy import copy, deepcopy
import json
import random

class Clobber(Game):
  def __init__(self, state, status, player):
    Game.__init__(self, state, status, player)

  def actions(self):
    return [] # this should return the possible actions

  def get_winner(self):
    return '.' # this should return the actual winner

  def other_player(self):
    if self.player == 'O': return 'X'
    if self.player == 'X': return 'O'

  def print_game(self):
    for row in self.get_board():
      print(row)


Instead of changing this code for a random solver, write new code that also returns the next move. For example, create a new "amazing_solver".

Solvers need to return a list or tuple of 3 elements: row, col and dir. Dir can be 'R', 'L', 'U', 'D'

In [5]:
def random_solver(game):
  board = game.get_board()
  player = game.player
  other_player = game.other_player()
  opts = []
  for row in range(len(board)):
    for col in range(len(board[row])):
      if board[row][col] == player:
        if row + 1 < len(board) and board[row + 1][col] == other_player:
          opts.append((row, col, 'D'))
        if col + 1 < len(board[row]) and board[row][col + 1] == other_player:
          opts.append((row, col, 'R'))
        if row - 1 >= 0 and board[row - 1][col] == other_player:
          opts.append((row, col, 'U'))
        if col - 1 >= 0 and board[row][col - 1] == other_player:
          opts.append((row, col, 'L'))
  return opts[random.randint(0, len(opts) - 1)]

This code is the one that starts the game. In different games you will need to change the class (Clobber is the one for clobber game).

The 'multi_player' variable can be True or False. If you set multi_palyer to True, you will need to provide a game_id against which to play or if using None a new game will be created (which another player can join).

In [6]:
game_loop(random_solver, Clobber, 'clobber', multi_player=False, id=None)

Creating new game...
Joining game with id: 28
Playing as O
Waiting for the other player to join...
['X', 'O', 'X', 'X', 'X']
['O', 'X', 'O', '.', 'O']
['X', 'O', 'X', 'O', 'X']
['O', 'X', 'O', 'X', 'O']
['X', 'O', 'X', 'O', 'X']
['O', 'X', 'O', 'X', 'O']
Making next move...
(1, 4, 'D')
b'Valid move'
['X', 'O', 'X', 'X', 'X']
['O', 'X', 'O', '.', '.']
['X', 'O', 'X', 'O', 'O']
['O', '.', 'O', 'X', 'O']
['X', 'X', 'X', 'O', 'X']
['O', 'X', 'O', 'X', 'O']
Making next move...
(2, 3, 'D')
b'Valid move'
['X', 'O', 'X', 'X', 'X']
['O', 'X', 'O', '.', '.']
['X', 'O', 'X', '.', 'O']
['O', '.', 'X', 'O', 'O']
['X', 'X', '.', 'O', 'X']
['O', 'X', 'O', 'X', 'O']
Making next move...
(5, 0, 'U')
b'Valid move'
['X', 'O', 'X', 'X', 'X']
['O', '.', 'X', '.', '.']
['X', 'O', 'X', '.', 'O']
['O', '.', 'X', 'O', 'O']
['O', 'X', '.', 'O', 'X']
['.', 'X', 'O', 'X', 'O']
Making next move...
(0, 1, 'R')
b'Valid move'
['X', '.', 'X', 'X', 'X']
['O', '.', '.', '.', '.']
['X', 'O', 'X', '.', 'O']
['O', '.', 'X',